In [104]:
import numpy as np
import open3d as o3d

def populate_gtboxes(sample_infos, dataset_name, classes, add_ground_lift=False):
    if dataset_name == 'nuscenes':
        zip_infos = zip(sample_infos['gt_boxes'], sample_infos['gt_names'], sample_infos['num_lidar_pts'])
    elif dataset_name in ['kitti', 'waymo','baraja']:
        anno = sample_infos['annos']
        annos_name = [name for name in anno['name']]
        zip_infos = zip(anno['gt_boxes_lidar'], annos_name, anno['num_points_in_gt'])
    else: 
        print(f"{dataset_name} is an unsupported dataset")
        return None

    pcd_gtboxes = {}        
    pcd_gtboxes['gt_boxes'], pcd_gtboxes['num_lidar_pts'], pcd_gtboxes['xyzlwhry_gt_boxes'] = [], [], []
    for idx, gt_anno in enumerate(zip_infos):
        bbox_corners, num_pts, xyzlwhry_bbox = get_o3dbox(gt_anno, classes=classes) 
        if bbox_corners != None:
            if add_ground_lift:
                bbox_corners.center = bbox_corners.center + [0,0,0.2] # Add 20cm to box centroid z-axis to get rid of the ground plane
            pcd_gtboxes['gt_boxes'].append(bbox_corners)
            pcd_gtboxes['num_lidar_pts'].append(num_pts)
            pcd_gtboxes['xyzlwhry_gt_boxes'].append(xyzlwhry_bbox)

    return pcd_gtboxes   

def get_o3dbox(anno_info, classes):
    """
    Convert from gt_boxes in the format [x,y,z,l,w,h,ry] to open3d oriented bbox
    which can then be used to crop a pointcloud
    """

    gt_box, class_name, num_lidar_pts = anno_info
    box_corners, r_mat = gtbox_to_corners(gt_box)
    boxpts = o3d.utility.Vector3dVector(box_corners)
    o3dbox = o3d.geometry.OrientedBoundingBox().create_from_points(boxpts)
    o3dbox.color = np.array([1,0,0])
    o3dbox.center = gt_box[0:3]
    o3dbox.R = r_mat
    return o3dbox, num_lidar_pts, gt_box

def gtbox_to_corners(box):
    """
    Takes an array containing [x,y,z,l,w,h,r], and returns an [8, 3] matrix that 
    represents the [x, y, z] for each 8 corners of the box.
    
    Note: Openpcdet __getitem__ gt_boxes are in the format [x,y,z,l,w,h,r,alpha]
    where alpha is "observation angle of object, ranging [-pi..pi]"
    """
    # To return
    corner_boxes = np.zeros((8, 3))

    translation = box[0:3]
    l, w, h = box[3], box[4], box[5] # waymo, nusc, kitti is all l,w,h after OpenPCDet processing
    rotation = box[6]

    # Create a bounding box outline
    bounding_box = np.array([
        [-l/2, -l/2, l/2, l/2, -l/2, -l/2, l/2, l/2],
        [w/2, -w/2, -w/2, w/2, w/2, -w/2, -w/2, w/2],
        [-h/2, -h/2, -h/2, -h/2, h/2, h/2, h/2, h/2]])

    # Standard 3x3 rotation matrix around the Z axis
    rotation_matrix = np.array([
        [np.cos(rotation), -np.sin(rotation), 0.0],
        [np.sin(rotation), np.cos(rotation), 0.0],
        [0.0, 0.0, 1.0]])

    return bounding_box.transpose(), rotation_matrix

def convert_to_o3dpcd(points):
    if type(points) == list:
        pcds = []
        for pointcloud in points:
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(pointcloud[:,:3])
            pcds.append(pcd)
        return pcds
    else:
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points[:,:3])
        return pcd


In [105]:
import pickle

fname = '/SEE-MTDA/data/nuscenes/custom_t4025-v3980/infos_meshednusc_GTBOX_MESH_MINPT60_MINUSCROPPED/nuscenes_infos_2sweeps_train_old.pkl'
with open(fname, 'rb') as f:
    mydata = pickle.load(f)

In [107]:
from tqdm import tqdm 

data_path = '/SEE-MTDA/data/nuscenes/custom_t4025-v3980/'
new_pkl = []

for idx, sample_infos in tqdm(enumerate(mydata), total=len(mydata)):
    data_pkl = sample_infos.copy()
    pcd_gtboxes = populate_gtboxes(sample_infos, 'nuscenes', 'car', add_ground_lift=False)        
    
    pc = np.fromfile(data_path + sample_infos['lidar_path'], dtype=np.float32).reshape((-1,5))[:,:3]
    meshed_pc = o3d.io.read_point_cloud(data_path + sample_infos['meshed_lidar_path'])
    o3dpcd = convert_to_o3dpcd(pc)
    
    data_pkl['num_meshed_lidar_pts'] = []
    print("len(mydata[23]['gt_boxes']) = ", len(sample_infos['gt_boxes']))
    print("len(pcd_gtboxes['gt_boxes']) = ", len(pcd_gtboxes['gt_boxes']))
    for gtbox in pcd_gtboxes['gt_boxes']:
        car = meshed_pc.crop(gtbox)
        data_pkl['num_meshed_lidar_pts'].append(len(car.points))
    
    data_pkl['num_meshed_lidar_pts'] = np.array(data_pkl['num_meshed_lidar_pts'])
        
    new_pkl.append(data_pkl)

100%|███████████████████████████████████████████| 4025/4025 [04:10<00:00, 16.08it/s]


In [108]:
len(new_pkl[23]['gt_names'])

56

In [109]:
len(new_pkl[23]['num_meshed_lidar_pts'])

56

In [113]:
save_fname = '/SEE-MTDA/data/nuscenes/custom_t4025-v3980/infos_meshednusc_GTBOX_MESH_MINPT60_MINUSCROPPED/nuscenes_infos_2sweeps_train.pkl'
with open(save_fname, 'wb') as f:
    pickle.dump(new_pkl, f)

In [114]:
import pickle

fname = '/SEE-MTDA/data/nuscenes/custom_t4025-v3980/infos_meshednusc_GTBOX_MESH_MINPT60_MINUSCROPPED/nuscenes_infos_2sweeps_train.pkl'
with open(fname, 'rb') as f:
    mydata2 = pickle.load(f)

In [116]:
len(mydata2[23]['gt_names'])

56